In [1]:
from nltk.util import ngrams
from nltk.util import bigrams
import nltk
from nltk.corpus import stopwords

import re
import pandas as pd
import numpy as np
import re
import operator

In [2]:
# reading ip data 
df = pd.read_excel("Merged_Apollo_Designations.xlsx")
print(len(df))
df.head()

25476


,Apollo Function,Apollo Division,Title
0,Consulting,Consultant,"Best Selling Author, Keynote Speaker and Strat..."
1,NaN,NaN,SAP S/4HANA Finance Managing Consultant
2,NaN,NaN,"Management Consultant, Author, Professional Sp..."
3,NaN,NaN,Consulting Professor
4,NaN,NaN,CEO / CMO / Strategist / Keynote Speaker / Con...


In [3]:
df['Title'] = df['Title'].replace(' ', np.nan)
df = df.dropna(axis=0, subset=['Title'])
print(len(df))

24644


In [4]:
df=df.ffill(axis = 0)
df.head()

,Apollo Function,Apollo Division,Title
0,Consulting,Consultant,"Best Selling Author, Keynote Speaker and Strat..."
1,Consulting,Consultant,SAP S/4HANA Finance Managing Consultant
2,Consulting,Consultant,"Management Consultant, Author, Professional Sp..."
3,Consulting,Consultant,Consulting Professor
4,Consulting,Consultant,CEO / CMO / Strategist / Keynote Speaker / Con...


In [5]:
def split_in_csv(word, no):
    word = str(word)
    k = word.split()
    n_gram = list(ngrams(k, n=no))
    return n_gram

def get_uni_gram(word):
    word = str(word)
    k = word.split()
    return k

In [16]:
def clean_text(text):
    lower = str(text).lower() 
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;&]')
    BAD_SYMBOLS_RE = re.compile('[^0-9A-Za-z #+_]')
    STOPWORDS = set(stopwords.words('english'))
    space_replace = REPLACE_BY_SPACE_RE.sub(" ",lower) #replace REPLACE_BY_SPACE_RE symbols by space in text
    nosymb = BAD_SYMBOLS_RE.sub("",space_replace) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = re.sub(r"\s*\b(?:{})\b".format("|".join(STOPWORDS)), "", nosymb) # delete STOPWORDS
    return re.sub(r" {2,}", " ", text.strip())

In [17]:
target_col_name='Title'
df[target_col_name] = df.apply(lambda x: clean_text(x[target_col_name]), axis = 1) ########

df['uni-gram'] = df.apply(lambda x: get_uni_gram(x[target_col_name]), axis = 1)
df['bi-gram'] = df.apply(lambda x: split_in_csv(x[target_col_name], 2), axis = 1)
df['tri-gram'] = df.apply(lambda x: split_in_csv(x[target_col_name], 3), axis = 1)

In [18]:
df.head()

,Apollo Function,Apollo Division,Title,uni-gram,bi-gram,tri-gram
0,Consulting,Consultant,best selling author keynote speaker strategic ...,"[best, selling, author, keynote, speaker, stra...","[(best, selling), (selling, author), (author, ...","[(best, selling, author), (selling, author, ke..."
1,Consulting,Consultant,sap 4hana finance managing consultant,"[sap, 4hana, finance, managing, consultant]","[(sap, 4hana), (4hana, finance), (finance, man...","[(sap, 4hana, finance), (4hana, finance, manag..."
2,Consulting,Consultant,management consultant author professional spea...,"[management, consultant, author, professional,...","[(management, consultant), (consultant, author...","[(management, consultant, author), (consultant..."
3,Consulting,Consultant,consulting professor,"[consulting, professor]","[(consulting, professor)]",[]
4,Consulting,Consultant,ceo cmo strategist keynote speaker consultant ...,"[ceo, cmo, strategist, keynote, speaker, consu...","[(ceo, cmo), (cmo, strategist), (strategist, k...","[(ceo, cmo, strategist), (cmo, strategist, key..."


In [19]:
fn_group=df.groupby('Apollo Function')
list_fn_groups = df['Apollo Function'].unique().tolist()

In [20]:
fn_group.first()

,Apollo Division,Title,uni-gram,bi-gram,tri-gram
Apollo Function,,,,,
Consulting,Consultant,best selling author keynote speaker strategic ...,"[best, selling, author, keynote, speaker, stra...","[(best, selling), (selling, author), (author, ...","[(best, selling, author), (selling, author, ke..."
Education,Teacher,bg5 foundations teacher,"[bg5, foundations, teacher]","[(bg5, foundations), (foundations, teacher)]","[(bg5, foundations, teacher)]"
Finance,Accounting,deputy finance accounting controller departmen...,"[deputy, finance, accounting, controller, depa...","[(deputy, finance), (finance, accounting), (ac...","[(deputy, finance, accounting), (finance, acco..."
Human Resources,Compensation & Benefits,benefits advisor,"[benefits, advisor]","[(benefits, advisor)]",[]
Information Technology,eCommerce Development,senior career coach linkedin trainer equity di...,"[senior, career, coach, linkedin, trainer, equ...","[(senior, career), (career, coach), (coach, li...","[(senior, career, coach), (career, coach, link..."
Legal,Acquisitions,vice president talent acquisitions,"[vice, president, talent, acquisitions]","[(vice, president), (president, talent), (tale...","[(vice, president, talent), (president, talent..."
Marketing,Technical Marketing,director technical marketing sales,"[director, technical, marketing, sales]","[(director, technical), (technical, marketing)...","[(director, technical, marketing), (technical,..."
Medical & Health,Anesthesiology,physician recruiter anesthesia,"[physician, recruiter, anesthesia]","[(physician, recruiter), (recruiter, anesthesia)]","[(physician, recruiter, anesthesia)]"
Operations,Call Center,head contact center suite,"[head, contact, center, suite]","[(head, contact), (contact, center), (center, ...","[(head, contact, center), (contact, center, su..."


In [21]:
count=0
for i in list_fn_groups:
    temp=fn_group.get_group(i)
    print("****",i,"****")
    count+=len(temp['Apollo Division'].unique().tolist())
print(count)

**** Consulting ****
**** Sales ****
**** Operations ****
**** Medical & Health ****
**** Marketing ****
**** Education ****
**** Finance ****
**** Human Resources ****
**** Information Technology ****
**** Legal ****
192


In [22]:
def get_dicts(df):
    dict_uni={}
    for i in df['uni-gram']:
        for j in i:
            if j not in dict_uni:
                dict_uni[j]=1
            else:
                dict_uni[j]+=1
    dict_uni = dict(sorted(  dict_uni.items(),
                            key=operator.itemgetter(1),
                            reverse=True))
    dict_bi={}
    for i in df['bi-gram']:
        i=list(map(" ".join, i))
        for j in i:
            if j not in dict_bi:
                dict_bi[j]=1
            else:
                dict_bi[j]+=1
    dict_bi = dict(sorted(  dict_bi.items(),
                            key=operator.itemgetter(1),
                            reverse=True))
    dict_tri={}
    for i in df['tri-gram']:
        i=list(map(" ".join, i))
        for j in i:
            if j not in dict_tri:
                dict_tri[j]=1
            else:
                dict_tri[j]+=1
    dict_tri = dict(sorted(  dict_tri.items(),
                            key=operator.itemgetter(1),
                            reverse=True))
    return dict_uni,dict_bi,dict_tri

In [23]:
for i in list_fn_groups:
    temp=fn_group.get_group(i)
    list_div_groups=temp['Apollo Division'].unique().tolist()
    for j in list_div_groups:
        list_Temp=temp.loc[temp['Apollo Division']==j]
        print("*",i,"*",j,"-->",len(list_Temp))

* Consulting * Consultant --> 125
* Sales * Account Management --> 125
* Sales * Business Development --> 125
* Sales * Channel Sales --> 125
* Sales * Customer Retention & Development --> 125
* Sales * Customer Success --> 125
* Sales * Field / Outside Sales --> 125
* Sales * Inside Sales --> 125
* Sales * Partnerships --> 125
* Sales * Revenue Operations --> 125
* Sales * Sales --> 125
* Sales * Sales Enablement --> 125
* Sales * Sales Engineering --> 125
* Sales * Sales Operations --> 125
* Sales * Sales Training --> 125
* Operations * Call Center --> 125
* Operations * Construction --> 125
* Operations * Corporate Strategy --> 125
* Operations * Customer Service / Support --> 125
* Operations * Enterprise Resource Planning --> 125
* Operations * Facilities Management --> 119
* Operations * Leasing --> 125
* Operations * Logistics --> 125
* Operations * Office Operations --> 125
* Operations * Operations --> 125
* Operations * Physical Securirty --> 125
* Operations * Project Develo

In [24]:
list_fn_groups

['Consulting',
 'Sales',
 'Operations',
 'Medical & Health',
 'Marketing',
 'Education',
 'Finance',
 'Human Resources',
 'Information Technology',
 'Legal']

In [25]:
data=pd.DataFrame()
ccc=0
for i in list_fn_groups:
    temp=fn_group.get_group(i)
    list_div_groups=temp['Apollo Division'].unique().tolist()
    for j in list_div_groups:
        list_Temp=temp.loc[temp['Apollo Division']==j]
        list_Temp.reset_index(inplace=True,drop=True)
        list_Temp=list_Temp[:125]
        toappend=[i,j]
        for k in list_Temp['Title'].tolist():
            toappend.append(k)
        while(len(toappend)<127):
            toappend.append(None)
        data[ccc]=toappend
        ccc+=1

In [15]:
# data.to_excel("Apollo_Func_Div_Title_wo_clean.xlsx")

In [17]:
# list_Temp.drop_duplicates(subset ="Title",
#                      keep = False, inplace = True)

In [26]:
un_dict_fin={}
bi_dict_fin={}
tri_dict_fin={}

for i in list_fn_groups:
    temp=fn_group.get_group(i)
    list_div_groups=temp['Apollo Division'].unique().tolist()
    for j in list_div_groups:
        list_Temp=temp.loc[temp['Apollo Division']==j]
        un_dict_fin[i+"-"+j],bi_dict_fin[i+"-"+j],tri_dict_fin[i+"-"+j]=get_dicts(list_Temp)

In [27]:
un_dict_fin['Legal-Privacy']

{'privacy': 146,
 'data': 47,
 'officer': 39,
 'security': 29,
 'compliance': 28,
 'manager': 25,
 'chief': 22,
 'information': 20,
 'director': 19,
 'senior': 19,
 'counsel': 17,
 'head': 14,
 'consultant': 13,
 'global': 13,
 'cybersecurity': 11,
 'legal': 10,
 'cyber': 9,
 'protection': 9,
 'risk': 9,
 'associate': 9,
 'governance': 8,
 'product': 7,
 'management': 7,
 'general': 7,
 'lead': 7,
 'marketing': 6,
 'partner': 6,
 'analyst': 6,
 'managing': 5,
 'specialist': 5,
 'sr': 5,
 'dpo': 5,
 'president': 4,
 'group': 4,
 'digital': 4,
 'program': 3,
 'trust': 3,
 'grc': 3,
 'vice': 3,
 'ciso': 3,
 'principal': 3,
 'strategy': 3,
 'technology': 3,
 'planning': 3,
 'cofounder': 3,
 'staff': 3,
 'university': 3,
 'office': 3,
 'advisory': 3,
 'innovation': 3,
 'health': 3,
 'safety': 2,
 'business': 2,
 'apac': 2,
 'regulatory': 2,
 'vp': 2,
 'gdpr': 2,
 'customer': 2,
 'services': 2,
 'advisor': 2,
 'lawyer': 2,
 'advocaat': 2,
 'energy': 2,
 'commercial': 2,
 'austria': 2,
 'trai

In [28]:
bi_dict_fin['Legal-Privacy']

{'privacy officer': 31,
 'data privacy': 29,
 'compliance privacy': 14,
 'security privacy': 13,
 'chief privacy': 13,
 'cybersecurity privacy': 10,
 'senior manager': 10,
 'privacy compliance': 9,
 'head data': 8,
 'privacy data': 8,
 'information security': 8,
 'data protection': 8,
 'general counsel': 7,
 'privacy consultant': 6,
 'governance privacy': 5,
 'security compliance': 5,
 'privacy lead': 5,
 'information privacy': 5,
 'officer dpo': 5,
 'manager privacy': 5,
 'global privacy': 4,
 'protection officer': 4,
 'privacy counsel': 4,
 'global data': 4,
 'data governance': 4,
 'associate general': 4,
 'manager data': 4,
 'program manager': 3,
 'cyber security': 3,
 'privacy grc': 3,
 'vice president': 3,
 'counsel privacy': 3,
 'director information': 3,
 'chief information': 3,
 'counsel chief': 3,
 'privacy specialist': 3,
 'privacy risk': 3,
 'privacy security': 3,
 'university legal': 3,
 'legal counsel': 3,
 'counsel information': 3,
 'manager cybersecurity': 3,
 'privacy a

In [29]:
tri_dict_fin['Legal-Privacy']

{'chief privacy officer': 11,
 'data privacy compliance': 5,
 'information security privacy': 5,
 'information privacy officer': 5,
 'privacy data protection': 5,
 'head data privacy': 4,
 'data protection officer': 4,
 'security compliance privacy': 4,
 'global data privacy': 4,
 'head data governance': 4,
 'associate general counsel': 4,
 'manager data privacy': 4,
 'compliance privacy officer': 4,
 'cyber security privacy': 3,
 'data privacy consultant': 3,
 'director information security': 3,
 'general counsel chief': 3,
 'counsel chief privacy': 3,
 'university legal counsel': 3,
 'legal counsel information': 3,
 'counsel information privacy': 3,
 'senior manager cybersecurity': 3,
 'manager cybersecurity privacy': 3,
 'compliance privacy analyst': 3,
 'cyber data privacy': 3,
 'global compliance privacy': 3,
 'privacy program manager': 2,
 'security privacy officer': 2,
 'managing director cofounder': 2,
 'compliance privacy specialist': 2,
 'privacy risk management': 2,
 'senior

In [43]:
data=pd.DataFrame()
ccc=0
for i in list_fn_groups:
    temp=fn_group.get_group(i)
    list_div_groups=temp['Apollo Division'].unique().tolist()
    for j in list_div_groups:
        list_Temp=temp.loc[temp['Apollo Division']==j]
        list_Temp.reset_index(inplace=True,drop=True)
        list_Temp=list_Temp[:125]
        toappend=[i,j]
        count=[None,None]
        un_dict_fin[i+"-"+j],bi_dict_fin[i+"-"+j],tri_dict_fin[i+"-"+j]=get_dicts(list_Temp)
        #print(i, " ",j)
        c=0
        for keys,val in un_dict_fin[i+"-"+j].items():
            toappend.append(keys)
            count.append(val)
            c+=1
            if(c==20):
                break
        while(c<20):
            toappend.append(None)
            count.append(None)
            c+=1
            
        c=0
        for keys,val in bi_dict_fin[i+"-"+j].items():
            toappend.append(keys)
            count.append(val)
            c+=1
            if(c==20):
                break
        while(c<20):
            toappend.append(None)
            count.append(None)
            c+=1
        c=0
        for keys,val in tri_dict_fin[i+"-"+j].items():
            toappend.append(keys)
            count.append(val)
            c+=1
            if(c==20):
                break
        while(c<20):
            toappend.append(None)
            count.append(None)
            c+=1
        data[ccc]=toappend
        ccc+=1
        data[ccc]=count
        ccc+=1
        print(len(toappend))

62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62


In [44]:
data.to_excel("Apollo_Div_N_grams_Sorted_Count.xlsx")

In [45]:
ccc

384